In [ ]:
# Import the required libraries

import os
import cv2
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from mahotas.features import surf as SURF

In [ ]:
DATA = "data"
PATH = os.path.abspath('')
SAMPLES = os.listdir(f'{PATH}/{DATA}')

SIFT = cv2.SIFT_create(nfeatures=400)
HOG = cv2.HOGDescriptor()

In [ ]:
def get_descriptors(sample, tech):
    images = os.listdir(f"{PATH}/{DATA}/{sample}")
    images = [f"{PATH}/{DATA}/{sample}/{name}" for name in images]
    descriptors = []
    
    for n, img in enumerate(images):
        img_rgb = cv2.imread(img)
        if tech == 'sift':
            img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
            kps, des = SIFT.detectAndCompute(img_gray, None)
            des = np.mean(des, axis=0)
        elif tech == 'surf':
            img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
            des = SURF.surf(img_gray)
            des = np.mean(des, axis=0)
        elif tech == 'hog':
            img_rgb = cv2.resize(img_rgb, (128, 128))
            img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
            des = HOG.compute(img_gray)
        descriptors.append(des)        
    return descriptors

In [ ]:
# Computing the descriptors

Data = {'sift': None, 'surf': None, 'hog': None}

for model in Data:
    data = []

    for sample in SAMPLES:
        desc = get_descriptors(sample, model)
        data += desc

    Data[model] = np.array(data)
    
    print("Data extracted successfully for", model.upper())

In [ ]:
# Labels for the DataSet

labels = np.zeros(606)

labels[0:202] = 0
labels[202:404] = 1
labels[404:606] = 2

In [ ]:
# Splitting DataSet into training (70%) and testing (30%)

DataSet = {'sift': {}, 'surf': {}, 'hog': {}}

for model in DataSet:
    DataSet[model]['x_train'], DataSet[model]['x_test'], DataSet[model]['y_train'], DataSet[model]['y_test'] = train_test_split(Data[model], labels, test_size=0.3)

In [ ]:
# Initializing the models

Models = {'sift': {}, 'surf': {}, 'hog': {}}

for model in Models:
    Models[model]['RFC'] = RandomForestClassifier(n_estimators=500)
    Models[model]['ANN'] = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', max_iter=1000)


In [ ]:
# Training RandomForestClassifiers

for model in Models:
    Models[model]['RFC'].fit(DataSet[model]['x_train'], DataSet[model]['y_train'])

In [ ]:
# Training ANNs

for model in Models:
    Models[model]['ANN'].fit(DataSet[model]['x_train'], DataSet[model]['y_train'])

In [ ]:
# Calculating the accuracy for all models

for model in Models:
    for i in Models[model]:
        pred = Models[model][i].predict(DataSet[model]['x_test'])
        acc = accuracy_score(DataSet[model]['y_test'], pred)
        
        print(f"{i} Accuracy ({model.upper()}):", acc)
    print()